In [1]:
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import torchvision as tv
import torchvision.transforms as transforms
import numpy as np
from PIL import Image



In [2]:
# 超参数设置
EPOCH = 10   #遍历数据集次数
BATCH_SIZE = 64      #批处理尺寸(batch_size)
LR = 0.001        #学习率
 
# 定义数据预处理方式
transform = transforms.ToTensor()
 
# 定义训练数据集
trainset = tv.datasets.MNIST(root='/Users/lichen/Downloads/DataSets/',
                             train=True,
                             download= False,
                             transform=transform)

# 定义训练批处理数据
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True,
    )
 
# 定义测试数据集
testset = tv.datasets.MNIST(root='/Users/lichen/Downloads/DataSets/',
                            train=False,
                            download=False,
                            transform=transform)
 
# 定义测试批处理数据
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=False,
    )

In [ ]:
for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

In [ ]:
np.shape(labels)

In [36]:
# 定义损失函数loss function 和优化方式（采用SGD）
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = fft_conv2d().to(device)
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数，通常用于多分类问题上
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)

In [35]:
import torch 
import torch.nn as nn
import torch.nn.functional as F 
import numpy as np
from PIL import Image


class fft_conv2d(nn.Module):

    def __init__(self, tile_size = 40, kernel_size = 32, tiling_factor = 4):
        super(fft_conv2d, self).__init__()
        self.tile_size = tile_size
        self.kernel_size = kernel_size
        self.tiling_factor = tiling_factor

        pad_one, pad_two = int(np.ceil((self.tile_size - self.kernel_size)/2)), int(np.floor((self.tile_size - self.kernel_size)//2))
        # self.kernels_lists = [[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0 ,0 ,0]]
        # for i in range(4):
        #     for j in range(4):
        #         names['kernels_' + str(i) + str(j)] = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))
        kernels_lists = [[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0 ,0 ,0]]
        
        for i in range(self.tiling_factor):
            for j in range(self.tiling_factor):
                # self.names['kernels_' + str(i) + str(j)] = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))
                exec('self.kernel_{}{} = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))'.format(i,j))
                exec('kernels_lists[i][j] = self.kernel_{}{}'.format(i,j))
        self.kernels_lists = kernels_lists
        kernels_pad =[[F.pad(kernel, (0,0,pad_one,pad_two, pad_one,pad_two), 'constant', 0) for kernel in kernels]for kernels in                                    self.kernels_lists]
        self.psf_pad = torch.cat([torch.cat(kernel_list, dim=0) for kernel_list in kernels_pad], dim=1)
        self.fc1 = nn.Linear(32, 16)

        

    def forward(self, x):
        x = self.fftconv2d(x, self.psf_pad, otf=None, adjoint=False, phase=True)
        x = self.MaxPool2d_complex(x)
        x = x.view(-1, 4 * 8)
        x = self.fc1(x)
        # nn.MaxPool2d(40, 40)
        # x = x.view(x.size(0), 256 * 6 * 6)
        # x = self.classifier(x)
        return x
    def MaxPool2d_complex(self, a_tensor):
        # shape = (1,1,160,160,2)
        a_r, a_j = a_tensor.split(1,-1)
        a_r = a_r.squeeze(-1)
        a_j = a_j.squeeze(-1)
        pool = nn.MaxPool2d(40)
        a_r_p = pool(a_r)
        a_j_p= pool(a_j)
        a_p = torch.stack([a_r_p, a_j_p], dim = -1)
        
        return a_p
        
    def torch_conj(self, a_tensor):

        # 复共轭
        otf_conj = torch.cat([a_tensor.index_select(-1, torch.tensor([0])), a_tensor.index_select(-1, torch.tensor([1])) * -1],dim = -1)
        return otf_conj

    def shift2d(self,a_tensor):

        x_shift = (np.shape(a_tensor)[0]+1)//2
        y_shift = (np.shape(a_tensor)[1]+1)//2
        shift2d = torch.roll(a_tensor,shifts = (x_shift, y_shift), dims = (0,1))
        return shift2d

    def ishift2d(self, a_tensor):

        x_shift = (np.shape(a_tensor)[0]+1)//2
        y_shift = (np.shape(a_tensor)[1]+1)//2
        ishift2d = torch.roll(a_tensor,shifts = (-x_shift, -y_shift), dims = (0,1))
        return ishift2d

    def psf2otf(self, psf, psf_size):

        # psf = F.pad()
        psf = self.ishift2d(psf)
        otf = torch.fft(psf, signal_ndim = 2)
        return otf

    def fftconv2d(self, img, psf_pad, otf=None, adjoint=False, phase=True):
        # real2complex
        # if len(img.shape) == 4:
        if img.shape[-1] != 2:
            img_j = torch.zeros_like(img)
            img = torch.stack([img, img_j], dim=-1)

        pad_one, pad_two = int(np.ceil((self.tile_size * self.tiling_factor  - img.shape[-2])/2)), int(np.floor((self.tile_size * self.tiling_factor - img.shape[-3])//2))
        img_pad = F.pad(img,(0, 0, pad_one, pad_two, pad_one, pad_two), "constant", 0)          
        
        img_pad_fft2d = torch.fft(img_pad, signal_ndim = 2)
        
        otf = self.psf2otf(psf_pad, psf_size = self.tile_size * self.tiling_factor)
        otf = otf.unsqueeze(0)
        
        if adjoint:
            result = torch.ifft(img_pad_fft2d * self.torch_conj(otf), signal_ndim = 2)
        else:
            result = torch.ifft(img_pad_fft2d * otf, signal_ndim = 2)

        if phase:  
            result = result
        else:
            result = index_select(-1, torch.tensor([0]))
        return result



In [5]:
for epoch in range(EPOCH):
        sum_loss = 0.0
        # 数据读取
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 梯度清零
            optimizer.zero_grad()
 
            # forward + backward
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
 
            # 每训练100个batch打印一次平均loss
            sum_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %d] loss: %.03f'
                      % (epoch + 1, i + 1, sum_loss / 100))
                sum_loss = 0.0
        # 每跑完一次epoch测试一下准确率
        with torch.no_grad():
            correct = 0
            total = 0
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                # 取得分最高的那个类
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, (100 * correct / total)))


RuntimeError: The size of tensor a (28) must match the size of tensor b (160) at non-singleton dimension 3

In [37]:
layer = fft_conv2d()
input = torch.randn((12,1,160,160),requires_grad = True)

In [38]:
output = layer(input)

UnboundLocalError: local variable 'img_pad' referenced before assignment

In [32]:
np.shape(output)

# img_pad_fft2d = torch.fft(ima_com, signal_ndim = 2)

torch.Size([12, 32])

torch.Size([12, 3, 20, 20, 2])

In [ ]:
otf_conj = torch.cat([a_tensor.index_select(-1, torch.tensor([0])), a_tensor.index_select(-1, torch.tensor([1])) * -1],dim = -1)


In [ ]:
kernels_lists = layer.psf_pad



In [73]:
a= torch.randn(32,32,2)
a_r, a_j = a.split(1,-1)
# b = torch.transpose(a, -2,-1)
# c = torch.transpose(b, -2,-3)

In [74]:
a_r= a_r.squeeze(-1)


In [75]:
np.shape(a_r)



torch.Size([32, 32])

In [ ]:
for name, parameter in layer.named_parameters():
    print(name,':',parameter)

In [ ]:
tile_size = 40
kernel_size = 32
tiling_factor = 4
pad_one, pad_two = int(np.ceil((tile_size - kernel_size)/2)), int(np.floor((tile_size - kernel_size)//2))
kernels_lists = [[nn.Parameter(torch.rand(kernel_size, kernel_size, 2)) for i in range(tiling_factor)]for j in range(tiling_factor)]
kernels_pad=[[F.pad(kernel, (0,0,pad_one,pad_two, pad_one,pad_two), 'constant', 0) for kernel in kernels]for kernels in kernels_lists]

psf = torch.cat([torch.cat(kernel_list, dim=0) for kernel_list in kernels_pad], dim=1)

psf_s = torch.fft(psf, signal_ndim = 2)

x_shift = (np.shape(psf_s)[0]+1)//2
y_shift = (np.shape(psf_s)[1]+1)//2

psf_s_shift = torch.roll(psf_s, shifts = (x_shift, y_shift), dims = (0,1))

# complex2real



In [ ]:
# transpose (0, 1) dimentions as H and W
tile_size = 40
kernel_size = 32
tiling_factor = 4

def torch_conj(a_tensor):

    # 复共轭
    otf_conj = torch.cat([a_tensor.index_select(-1, torch.tensor([0])), a_tensor.index_select(-1, torch.tensor([1])) * -1],dim = -1)
    return otf_conj

def shift2d(a_tensor):

    x_shift = (np.shape(a_tensor)[0]+1)//2
    y_shift = (np.shape(a_tensor)[1]+1)//2
    shift2d = torch.roll(a_tensor,shifts = (x_shift, y_shift), dims = (0,1))
    return shift2d

def ishift2d(a_tensor):

    x_shift = (np.shape(a_tensor)[0]+1)//2
    y_shift = (np.shape(a_tensor)[1]+1)//2
    ishift2d = torch.roll(a_tensor,shifts = (-x_shift, -y_shift), dims = (0,1))
    return ishift2d

def ptf2otf(psf, psf_size):

    # psf = F.pad()
    psf = ishift2d(psf)
    otf = torch.fft(tmp, signal_ndim = 2)
    return otf

def fft_conv2d(img_pad, psf_pad, otf=None, adjoint=False, phase=True):
    # real2complex
    img_pad_fft2d = torch.rfft(img_pad, signal_ndim = 2)
    
    otf = ptf2otf(psf_pad, psf_size = tile_size * tiling_factor)
    
    if adjoint:
        result = torch.ifft(img_fft * torch_conj(otf), signal_ndim = 2)
    else:
        result = torch.ifft(img_fft * otf, signal_ndim = 2)

    if phase:  
        result = result
    else:
        result = index_select(-1, torch.tensor([0]))

    return result



In [ ]:
e = torch.tensor([0, 2])

In [ ]:
c = torch.rand(5,5,2)
d = c.index_select(-1, torch.tensor([1])) * -1
e = torch.cat([c.index_select(-1, torch.tensor([0])), c.index_select(-1, torch.tensor([1])) * -1],dim = -1)


In [ ]:
a = nn.Parameter(torch.rand(5,5))

a_fft = torch.rfft(a, signal_ndim = 2)

a_shift = shift2d(a_fft)
a_ishift = ishift2d(a_shift)

In [ ]:
# test fft and shift
a = nn.Parameter(torch.rand(5,5,2))
print(a)
a_s = torch.fft(a, signal_ndim = 2)
print(a_s)
x_shift = (np.shape(a_s)[0]+1)//2
y_shift = (np.shape(a_s)[1]+1)//2
a_shift = torch.roll(a_s, shifts = (x_shift, y_shift), dims = (0,1))
print(a_s)

In [ ]:
# test image Fourier transform
Ima = Image.open('3.bmp')
Ima = Ima.convert('L')
Ima_g = np.array(Ima)
print(np.shape(Ima_g))
Ima_g = torch.from_numpy(Ima_g)
Ima_g = Ima_g.float()
print(np.shape(Ima_g))
Ima_r = torch.FloatTensor(Ima_g)
Ima_j = torch.zeros(256,256)
Ima_com = torch.cat([Ima_r.unsqueeze(-1), Ima_j.unsqueeze(-1)], dim=2)
Ima_s = torch.fft(Ima_com, signal_ndim = 2)
print(np.shape(Ima_s))
x_shift = (np.shape(Ima_s)[0]+1)//2
y_shift = (np.shape(Ima_s)[1]+1)//2
Ima_shift = torch.roll(Ima_s, shifts = (x_shift, y_shift), dims = (0,1))
print(np.shape(Ima_shift))
Ima_shift_np = np.array(Ima_shift)
Ima_shift_np = np.sqrt(np.square(Ima_shift_np[:,:,0])+np.square(Ima_shift_np[:,:,1]))
# Ima_png = np.abs(Ima_shift)
# Ima_png = Ima_png.numpy()
img = Image.fromarray(Ima_shift_np)
img.show()

In [ ]:
kernel = [[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0 ,0 ,0]]
for i in range(4):
    for j in range(4):
        kernel[i][j] = nn.Parameter(torch.rand(40, 40, 2))       

In [ ]:
class fft_conv2d(nn.Module):

    def __init__(self, tile_size, kernel_size, tiling_factor):
        names = self.__dict__
        super(fft_conv2d, self).__init__()
        self.names = names
        self.tile_size = tile_size
        self.kernel_size = kernel_size
        self.tiling_factor = tiling_factor
        # self.w = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))
        pad_one, pad_two = int(np.ceil((self.tile_size - self.kernel_size)/2)), int(np.floor((self.tile_size - self.kernel_size)//2))

        self.kernels_lists = [[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0, 0 ,0],[0, 0 ,0 ,0]]
        
        for i in range(self.tiling_factor):
            for j in range(self.tiling_factor):
                # self.names['kernels_' + str(i) + str(j)] = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))
                exec('self.kernel_{}{} = nn.Parameter(torch.rand(self.kernel_size, self.kernel_size, 2))'.format(i,j))
                exec('self.kernels_lists[i][j] = self.kernel_{}{}'.format(i,j))
        kernels_pad =[[F.pad(kernel, (0,0,pad_one,pad_two, pad_one,pad_two), 'constant', 0) for kernel in kernels]for kernels in                                    self.kernels_lists]
        self.psf_pad = torch.cat([torch.cat(kernel_list, dim=0) for kernel_list in kernels_pad], dim=1)
        

In [ ]:
fff = fft_conv2d(40 ,32 ,4)

In [ ]:
a = fff.psf_pad

In [ ]:
np.shape(a)

In [ ]:
for name, parameter in fff.named_parameters():
    print(name,':',parameter)

In [ ]:
for i in range(2):

    for j in range(2):
        b = torch.randn(5,5)
        if j > 0:
            b = torch.cat([b, b], dim = )
             
        


In [ ]:
b = torch.randn(5,5)
c= b.unsqueeze(0).unsqueeze(0)

np.shape(b)
np.shape(c)


In [ ]:
torch.cat([c,b],dim =-1)

In [ ]:
a = Linear

In [ ]:
torch.mm()

In [ ]:
class Linear(nn.Module):
    pass